<h3> Making the Classification Model on custom dataset </h3>

<p> Make Classification data </p>

In [ ]:
from sklearn.datasets import make_circles

n_samples = 10000

X, y = make_circles(n_samples=n_samples, factor=.3, noise=.05)

In [ ]:
print(f"First 10 Features: {X[:10]}")
print(f"First 10 Labels: {y[:10]}")

In [ ]:
import pandas as pd

circles = pd.DataFrame({"X0": X[:, 0], "X1": X[:, 1], "label": y})

circles.head(10)

In [ ]:
circles.label.value_counts()

<p> Let's plot them. </p>

```python


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x=circles.X0, y=circles.X1, c=circles.label, cmap=plt.cm.RdYlBu)
plt.show()

<p> Onto the model! </p>

```python

In [ ]:
# check the shapes
X.shape, y.shape

In [ ]:
X_sample = X[0]
y_sample = y[0]
print(f"Values for one sample of X: {X_sample} and the same for y: {y_sample}")
print(f"Shape for one sample of X: {X_sample.shape} and the same for y: {y_sample.shape}")

In [ ]:
import torch

X = torch.from_numpy(X).type(torch.float32)
y = torch.from_numpy(y).type(torch.float32)

X[:10], y[:10]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

len(X_train), len(X_test), len(y_train), len(y_test)

## Building the model

In [ ]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class CircleModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=2, out_features=5)
        self.layer_2 = nn.Linear(in_features=5, out_features=1)

    def forward(self, x):
        x = self.layer_2(torch.relu(self.layer_1(x)))
        return x

model_0 = CircleModelV0().to(device)
model_0

In [ ]:
# Make predictions with the model
untrained_preds = model_0(X_test.to(device))
print(f"Length of predictions: {len(untrained_preds)}, Shape: {untrained_preds.shape}")
print(f"Length of test samples: {len(y_test)}, Shape: {y_test.shape}")
print(f"\nFirst 10 Predictions: {untrained_preds[:10]}")
print(f"\nFirst 10 Actual labels: {y_test[:10]}")

### Setup loss function and optimizer


In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model_0.parameters(), lr=0.01)

In [ ]:
def accuracy_fn(y_true, y_preds):
    correct = torch.eq(y_true, y_preds).sum().item()
    accuracy = (correct / len(y_true)) * 100
    return accuracy

In [ ]:
y_logits = model_0(X_test.to(device))[:5]
y_logits

In [ ]:
y_pred_probs = torch.sigmoid(y_logits)
y_pred_probs

In [ ]:
y_preds = torch.round(y_pred_probs)

y_pred_labels = torch.round(torch.sigmoid(model_0(X_test.to(device))[:5]))

print(torch.eq(y_preds.squeeze(), y_pred_labels.squeeze()))

y_preds.squeeze()

In [ ]:
torch.manual_seed(42)

epochs = 100

X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(epochs):
    model_0.train()

    y_logits = model_0(X_train).squeeze()
    y_preds = torch.round(torch.sigmoid(y_logits))

    train_loss = loss_fn(y_logits, y_train)
    train_acc = accuracy_fn(y_train, y_preds)

    optimizer.zero_grad()

    train_loss.backward()

    optimizer.step()

    model_0.eval()

    with torch.inference_mode():
        test_logits = model_0(X_test).squeeze()
        test_preds = torch.round(torch.sigmoid(test_logits))

        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_test, test_preds)
    
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"\t\tTest Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%\n")


In [ ]:
import requests
from pathlib import Path 

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

from helper_functions import plot_predictions, plot_decision_boundary

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Training")
plot_decision_boundary(model_0, X=X_train, y=y_train)
plt.subplot(1, 2, 2)
plt.title("Test")
plot_decision_boundary(model_0, X=X_test, y=y_test)

Actually i'm following this tutorial and apparently i shouldn't have added a relu layer. So, my model performed better but the tutorial goes on to improve the previous model i.e. CircleModelV0. So, i'm going to do the same. 

```python

In [ ]:
class CircleModelV1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=2, out_features=10)
        self.layer_2 = nn.Linear(in_features=10, out_features=5)
        self.layer_3 = nn.Linear(in_features=5, out_features=1)

    def forward(self, x):
        x = self.layer_3(torch.relu(self.layer_2(torch.relu(self.layer_1(x)))))
        return x

model_1 = CircleModelV1().to(device)
model_1

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model_1.parameters(), lr=0.01)

In [ ]:
torch.manual_seed(42)

epochs = 1000

X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(epochs):
    model_1.train()

    y_logits = model_1(X_train).squeeze()
    y_preds = torch.round(torch.sigmoid(y_logits))

    loss = loss_fn(y_logits, y_train)
    accuracy = accuracy_fn(y_train, y_preds)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model_1.eval()

    with torch.inference_mode():
        test_logits = model_1(X_test).squeeze()
        test_preds = torch.round(torch.sigmoid(test_logits))

        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_test, test_preds)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train Loss: {loss:.4f} | Train Acc: {accuracy:.2f}%")
        print(f"\t\tTest Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%\n")


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Training")
plot_decision_boundary(model_1, X=X_train, y=y_train)
plt.subplot(1, 2, 2)
plt.title("Test")
plot_decision_boundary(model_1, X=X_test, y=y_test)

## Checking if the model can model a straight line

In [ ]:
weight = 0.7
bias = 0.3
start = 0
end = 1
step = 0.001

X_regression = torch.arange(start, end, step).unsqueeze(1)
y_regression = weight * X_regression + bias

print(len(X_regression), len(y_regression))
X_regression[:10], y_regression[:10]

In [ ]:
# Create train and test splits
train_split = int(0.8 * len(X_regression)) # 80% of data for training
X_train_regression, y_train_regression = X_regression[:train_split], y_regression[:train_split]
X_test_regression, y_test_regression = X_regression[train_split:], y_regression[train_split:]

print(len(X_train_regression), len(y_train_regression))
print(len(X_test_regression), len(y_test_regression))

In [ ]:
plot_predictions(train_data=X_train_regression, train_labels=y_train_regression, test_data=X_test_regression, test_labels=y_test_regression)

In [ ]:
model_2 = nn.Sequential(
    nn.Linear(in_features=1, out_features=10),
    nn.Linear(in_features=10, out_features=10),
    nn.Linear(in_features=10, out_features=1)
).to(device)

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model_2.parameters(), lr=0.01)

In [ ]:
torch.manual_seed(42)

epochs = 1000

X_train_regression, y_train_regression = X_train_regression.to(device), y_train_regression.to(device)
X_test_regression, y_test_regression = X_test_regression.to(device), y_test_regression.to(device)

for epoch in range(epochs):
    y_preds = model_2(X_train_regression)

    loss = loss_fn(y_preds, y_train_regression)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    model_2.eval()

    with torch.inference_mode():
        test_preds = model_2(X_test_regression)

        test_loss = loss_fn(test_preds, y_test_regression)
    
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train Loss: {loss:.4f}")
        print(f"\t\tTest Loss: {test_loss:.4f}\n")

In [ ]:
model_2.eval()

with torch.inference_mode():
    y_preds = model_2(X_test_regression)

plot_predictions(train_data=X_train_regression, train_labels=y_train_regression, test_data=X_test_regression, test_labels=y_test_regression, predictions=y_preds.cpu())

## For Non-linear data
Now i am supposed to use relu.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles

n_samples = 100000

X, y = make_circles(n_samples=n_samples, noise=.07)
plt.scatter(X[:, 0],X[:, 1], c=y, cmap=plt.cm.RdYlBu)

In [ ]:
import torch
from sklearn.model_selection import train_test_split

X = torch.from_numpy(X).type(torch.float32)
y = torch.from_numpy(y).type(torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

print(f"X_train: {len(X_train)} | y_train: {len(y_train)}")
print(f"X_test: {len(X_test)} | y_test: {len(y_test)}")
X_train[:10], y_train[:10]

In [ ]:
from torch import nn

class CircleModelV2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=2, out_features=10)
        self.layer_2 = nn.Linear(in_features=10, out_features=10)
        self.layer_3 = nn.Linear(in_features=10, out_features=1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

model_3 = CircleModelV2().to(device)
model_3

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model_3.parameters(), lr=0.01)

In [ ]:
torch.manual_seed(42)

epochs = 1000

X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(epochs):
    y_logits = model_3(X_train).squeeze()
    y_preds = torch.round(torch.sigmoid(y_logits))

    loss = loss_fn(y_logits, y_train)
    accuracy = accuracy_fn(y_train, y_preds)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model_3.eval() 

    with torch.inference_mode():
        test_logits = model_3(X_test).squeeze()
        test_preds = torch.round(torch.sigmoid(test_logits))

        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_test, test_preds)
    
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train Loss: {loss:.4f} | Train Acc: {accuracy:.2f}%")
        print(f"\t\tTest Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%\n")

In [ ]:
model_3.eval()
with torch.inference_mode():
    y_preds = torch.round(torch.sigmoid(model_3(X_test).squeeze()))
y_preds[:10], y_test


In [ ]:
# Plot decision boundaries for training and test sets
plt.figure(figsize=(12, 6))
plt.subplot(2, 4, 1)
plt.title("Train")
plot_decision_boundary(model_1, X_train, y_train) # model_1 = adam
plt.subplot(2, 4, 2)
plt.title("Test")
plot_decision_boundary(model_1, X_test, y_test)
plt.subplot(2, 4, 3)
plt.title("Train")
plot_decision_boundary(model_3, X_train, y_train)
plt.subplot(2, 4, 4)
plt.title("Test")
plot_decision_boundary(model_3, X_test, y_test) # model_3 = sgd